# 01 Obtaining Data

In [32]:
import pandas as pd
import warnings

## 1.1 Nat. Disasters - Importing Data
https://www.kaggle.com/datasets/brsdincer/all-natural-disasters-19002021-eosdis

In [33]:
raw_natural_disaster_df = pd.read_csv('data/all_natural_disasters.csv')

relevant_columns = ['Year', 'Disaster Subgroup', 'Disaster Type', 'Event Name', 'Country', 
'ISO', 'Region', 'Continent', 'Start Year', 'End Year', 'Total Deaths', 'Total Affected']

natural_disaster_df = raw_natural_disaster_df[relevant_columns]
natural_disaster_df.head()

,Year,Disaster Subgroup,Disaster Type,Event Name,Country,ISO,Region,Continent,Start Year,End Year,Total Deaths,Total Affected
0,1900,Climatological,Drought,NaN,Cabo Verde,CPV,Western Africa,Africa,1900,1900,11000.0,NaN
1,1900,Climatological,Drought,NaN,India,IND,Southern Asia,Asia,1900,1900,1250000.0,NaN
2,1902,Geophysical,Earthquake,NaN,Guatemala,GTM,Central America,Americas,1902,1902,2000.0,NaN
3,1902,Geophysical,Volcanic activity,Santa Maria,Guatemala,GTM,Central America,Americas,1902,1902,1000.0,NaN
4,1902,Geophysical,Volcanic activity,Santa Maria,Guatemala,GTM,Central America,Americas,1902,1902,6000.0,NaN


## 1.2 Nat. Disasters - Handling Columns

### 1.2.1 Start & End Year
Start and End Year Column will be transformed to a duration column. This way its easier to identy events that took place over a longer time period.

In [34]:
# Transforming Start and End Year Column to Duration
natural_disaster_df = natural_disaster_df.assign(Duration=(natural_disaster_df['End Year']-natural_disaster_df['Start Year']))
natural_disaster_df = natural_disaster_df.drop(['Start Year', 'End Year'], axis=1)
natural_disaster_df.head()

,Year,Disaster Subgroup,Disaster Type,Event Name,Country,ISO,Region,Continent,Total Deaths,Total Affected,Duration
0,1900,Climatological,Drought,NaN,Cabo Verde,CPV,Western Africa,Africa,11000.0,NaN,0
1,1900,Climatological,Drought,NaN,India,IND,Southern Asia,Asia,1250000.0,NaN,0
2,1902,Geophysical,Earthquake,NaN,Guatemala,GTM,Central America,Americas,2000.0,NaN,0
3,1902,Geophysical,Volcanic activity,Santa Maria,Guatemala,GTM,Central America,Americas,1000.0,NaN,0
4,1902,Geophysical,Volcanic activity,Santa Maria,Guatemala,GTM,Central America,Americas,6000.0,NaN,0


In [35]:
natural_disaster_df['Duration'].unique()

array([ 0,  4,  9,  2,  3,  6,  1,  5, 50], dtype=int64)

### 1.2.2 Disaster Subgroup & Type
- Subgroup becomes Group since Group contained the same value for each row (Natural)
- "Disaster" can be removed from the column names.

In [36]:
rename_dic = {'Disaster Subgroup' : 'Group', 'Disaster Type' : 'Type'}
natural_disaster_df = natural_disaster_df.rename(columns=rename_dic)
natural_disaster_df.head()

,Year,Group,Type,Event Name,Country,ISO,Region,Continent,Total Deaths,Total Affected,Duration
0,1900,Climatological,Drought,NaN,Cabo Verde,CPV,Western Africa,Africa,11000.0,NaN,0
1,1900,Climatological,Drought,NaN,India,IND,Southern Asia,Asia,1250000.0,NaN,0
2,1902,Geophysical,Earthquake,NaN,Guatemala,GTM,Central America,Americas,2000.0,NaN,0
3,1902,Geophysical,Volcanic activity,Santa Maria,Guatemala,GTM,Central America,Americas,1000.0,NaN,0
4,1902,Geophysical,Volcanic activity,Santa Maria,Guatemala,GTM,Central America,Americas,6000.0,NaN,0


## 1.3 Nat. Disasters - Display Dataframe Info

In [37]:
natural_disaster_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16126 entries, 0 to 16125
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Year            16126 non-null  int64  
 1   Group           16126 non-null  object 
 2   Type            16126 non-null  object 
 3   Event Name      3861 non-null   object 
 4   Country         16126 non-null  object 
 5   ISO             16126 non-null  object 
 6   Region          16126 non-null  object 
 7   Continent       16126 non-null  object 
 8   Total Deaths    11413 non-null  float64
 9   Total Affected  11617 non-null  float64
 10  Duration        16126 non-null  int64  
dtypes: float64(2), int64(2), object(7)
memory usage: 1.4+ MB


## 1.4 Nat. Disasters - Preparing for Merge

### 1.4.1 Dropping irrelevant Years
The Worldbank API only provides GDP's for the years 1960 to 2021. To test whether the events have an effect on the countries gdps the gdps from the year before, during as well as 3 years after the event will be used. That means that for this project only the years from 1961 to 2018 can be used.

In [38]:
drop_indexes = natural_disaster_df[(natural_disaster_df['Year'] < 1961)].index
drop_indexes = drop_indexes.append(natural_disaster_df[(natural_disaster_df['Year'] > 2018)].index)
nat_df = natural_disaster_df.drop(drop_indexes)

nat_df['Year'].unique()

array([1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
       1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982,
       1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
       1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018], dtype=int64)

In [39]:
# Fixing Index
nat_df = nat_df.reset_index(drop=True)
nat_df.head()

,Year,Group,Type,Event Name,Country,ISO,Region,Continent,Total Deaths,Total Affected,Duration
0,1961,Meteorological,Storm,NaN,Bangladesh,BGD,Southern Asia,Asia,11000.0,NaN,0
1,1961,Meteorological,Storm,NaN,Bangladesh,BGD,Southern Asia,Asia,NaN,NaN,0
2,1961,Meteorological,Storm,NaN,Bangladesh,BGD,Southern Asia,Asia,266.0,NaN,0
3,1961,Meteorological,Storm,Hattie,Belize,BLZ,Central America,Americas,275.0,NaN,0
4,1961,Climatological,Drought,NaN,Canada,CAN,Northern America,Americas,NaN,NaN,0


### 1.4.2 Obtaining List of ISO Codes

In [40]:
# Getting all GDP's once per country
country_iso_codes = nat_df['ISO'].unique().tolist()
len(country_iso_codes)

227

## 1.5 GDP's - Testing API

## Requests
- GDP Definition: https://api.worldbank.org/v2/indicator/NY.GDP.MKTP.CD
- Getting GDP all: https://api.worldbank.org/v2/country/all/indicator/NY.GDP.MKTP.CD?page=1
- Getting GDP by Country ISO Code: https://api.worldbank.org/v2/country/cpv/indicator/NY.GDP.MKTP.CD?per_page=62
- Getting GDP for Certain Years: https://api.worldbank.org/v2/country/cpv/indicator/NY.GDP.MKTP.CD?date=1960:1964

In [41]:
import requests
import xml.etree.ElementTree as ET
from genericpath import exists

### 1.5.1 Testing Request and Handling XML Response

In [42]:
response = requests.get('https://api.worldbank.org/v2/country/cpv/indicator/NY.GDP.MKTP.CD?date=2000:2001')

root = ET.fromstring(response.content)

for child in root:
    print(child.tag)
    for subchild in child:
        print('\t', subchild.tag, subchild.text)

{http://www.worldbank.org}data
	 {http://www.worldbank.org}indicator GDP (current US$)
	 {http://www.worldbank.org}country Cabo Verde
	 {http://www.worldbank.org}countryiso3code CPV
	 {http://www.worldbank.org}date 2001
	 {http://www.worldbank.org}value 563024383.296626
	 {http://www.worldbank.org}unit None
	 {http://www.worldbank.org}obs_status None
	 {http://www.worldbank.org}decimal 0
{http://www.worldbank.org}data
	 {http://www.worldbank.org}indicator GDP (current US$)
	 {http://www.worldbank.org}country Cabo Verde
	 {http://www.worldbank.org}countryiso3code CPV
	 {http://www.worldbank.org}date 2000
	 {http://www.worldbank.org}value 539227277.626411
	 {http://www.worldbank.org}unit None
	 {http://www.worldbank.org}obs_status None
	 {http://www.worldbank.org}decimal 0


In [43]:
for entry in root:
    print(entry.find('{http://www.worldbank.org}date').text, ' : ', entry.find('{http://www.worldbank.org}value').text)

2001  :  563024383.296626
2000  :  539227277.626411


## 1.6 GDP's - Obtaining GDP's for all Countries & Save

In [54]:
warnings.filterwarnings('ignore', category=FutureWarning)

# ONLY DO THIS STEP WHEN NECESSARY
# Duration ~ 45min
if (not exists('data/country_gdps.csv')):
    country_gpds_df = pd.DataFrame()
    error_isos = []

    # Iterate over iso codes and obtaining all gdp values for these countries
    for iso in country_iso_codes:
        try:
            url = f'https://api.worldbank.org/v2/country/{iso}/indicator/NY.GDP.MKTP.CD?per_page=62'
            response = requests.get(url)

            root = ET.fromstring(response.content)

            gdp_dict = {'iso' : iso}
            for entry in root:
                gdp_dict[entry.find('{http://www.worldbank.org}date').text] = entry.find('{http://www.worldbank.org}value').text

            country_gpds_df = country_gpds_df.append(gdp_dict, ignore_index=True)

        except:
            # Collecting list of iso codes the worldbank api does not list
            error_isos.append(iso)

    # Save to file
    error_df = pd.DataFrame(error_isos)
    error_df.to_csv('data/worldbank_iso_erros.csv')
    country_gpds_df.to_csv('data/country_gdps.csv')

else:
    country_gpds_df = pd.read_csv('data/country_gdps.csv', index_col=0)
    error_df = pd.read_csv('data/worldbank_iso_erros.csv', index_col=0)
    error_isos = error_df['0'].to_list()

country_gpds_df.head()


,iso,2021,2020,2019,2018,2017,2016,2015,2014,2013,...,1969,1968,1967,1966,1965,1964,1963,1962,1961,1960
0,BGD,4.162649e+11,3.739021e+11,3.512384e+11,3.213790e+11,2.937546e+11,2.652362e+11,1.950787e+11,1.728855e+11,1.499905e+11,...,8.471006e+09,7.483685e+09,7.253575e+09,6.439688e+09,5.906637e+09,5.386055e+09,5.319458e+09,5.081413e+09,4.817580e+09,4.274894e+09
1,BLZ,1.789923e+09,1.585632e+09,1.945250e+09,1.887465e+09,1.844907e+09,1.796929e+09,1.734320e+09,1.676407e+09,1.581845e+09,...,4.730539e+07,4.491018e+07,4.737931e+07,4.440559e+07,4.006993e+07,3.619383e+07,3.374941e+07,3.185692e+07,2.996437e+07,2.807189e+07
2,CAN,1.990762e+12,1.645423e+12,1.742015e+12,1.725329e+12,1.649266e+12,1.527995e+12,1.556509e+12,1.805750e+12,1.846597e+12,...,7.914841e+10,7.182981e+10,6.566866e+10,6.108838e+10,5.451518e+10,4.937752e+10,4.502999e+10,4.222745e+10,4.093495e+10,4.046172e+10
3,ETH,1.112711e+11,1.076577e+11,9.591259e+10,8.426935e+10,8.177079e+10,7.429662e+10,6.458933e+10,5.561223e+10,4.764821e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FRA,2.937473e+12,2.630318e+12,2.728870e+12,2.790957e+12,2.595151e+12,2.472964e+12,2.439189e+12,2.855964e+12,2.811877e+12,...,1.419031e+11,1.297854e+11,1.189730e+11,1.100459e+11,1.015372e+11,9.400785e+10,8.475920e+10,7.560753e+10,6.746164e+10,6.222548e+10


## 1.7 GDP's - Reviewing Errors
Taking a look at the countries the Wordlbank does not list GDPs for

In [55]:
nat_df[nat_df['ISO'].isin(error_isos)][['ISO', 'Country']].drop_duplicates()

,ISO,Country
16,DFR,Germany Fed Rep
558,SUN,Soviet Union


## 1.8 Nat. Disasters - Clean Up & Save
The Worldbank seems to not list GDP's for countries that do not exist anymore as well as countries that are extremely small with population sizes of less then 10000.

For now these countries are going to be excluded from the dataframe. The possibility remains to lookup the GDP manually for these countries should that be necessary.

In [ ]:
nat_df_reduced = nat_df.drop(nat_df[nat_df['ISO'].isin(error_isos)].index)
nat_df_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13834 entries, 0 to 14051
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Year            13834 non-null  int64  
 1   Group           13834 non-null  object 
 2   Type            13834 non-null  object 
 3   Event Name      3424 non-null   object 
 4   Country         13834 non-null  object 
 5   ISO             13834 non-null  object 
 6   Region          13834 non-null  object 
 7   Continent       13834 non-null  object 
 8   Total Deaths    9706 non-null   float64
 9   Total Affected  10300 non-null  float64
 10  Duration        13834 non-null  int64  
dtypes: float64(2), int64(2), object(7)
memory usage: 1.3+ MB


In [ ]:
nat_df_reduced.to_csv('data/all_natural_disasters_reduced.csv')